<a href="https://colab.research.google.com/github/rhaguirrem/ColorEvo/blob/master/Anterpolator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.spatial import cKDTree
import random
from IPython.display import display
import os
from google.colab import files
#from tqdm import tqdm
from tqdm.notebook import tqdm
import gc
import copy
from math import copysign

# Define the offsets for the 26 neighboring cells
OFFSETS = [(i, j, k) for i in [-1, 0, 1] for j in [-1, 0, 1] for k in [-1, 0, 1] if (i, j, k) != (0, 0, 0)]

def round_half_up(n):
    return int(n + 0.5 * np.sign(n))

def sign(a):
    return copysign(1,a)

class Grid:
    def __init__(self, grid_dimension, lower_bounds, cell_size, samples_df):
        self.grid_dimension = grid_dimension
        self.cell_size = cell_size
        self.lower_bounds = lower_bounds
        self.cells = np.empty(grid_dimension, dtype=object)

        empty_cell_template = GridCell(None, pd.DataFrame(), ant_capacity=100)
        for i in tqdm(range(grid_dimension[0]), desc='Initializing empty grid...'):
            for j in range(grid_dimension[1]):
                for k in range(grid_dimension[2]):
                    centroid = tuple(self.lower_bounds[d] + (idx + 0.5) * self.cell_size[d] for d, idx in enumerate([i, j, k]))
                    new_cell = copy.deepcopy(empty_cell_template)
                    new_cell.position = centroid
                    self.cells[i, j, k] = new_cell#GridCell(centroid, pd.DataFrame(), ant_capacity=100)
        self.samples_df = samples_df
        self.domained = False
        self.out_of_grid_cells = set()

        # Build a KDTree from the sample points
        self.kdtree = cKDTree(self.samples_df[['x', 'y', 'z']].values)

    def initialize_regular_grid(self):
        assigned_samples = set()
        for index, row in tqdm(self.samples_df.iterrows(), desc='Assigning samples to cells...'):
            # Skip this sample if it has already been assigned to a cell
            if index in assigned_samples:
                continue
            # Get the cell indices from the 'grid_indices' column
            i, j, k = row['grid_indices']
            centroid = tuple(self.lower_bounds[d] + (idx + 0.5) * self.cell_size[d] for d, idx in enumerate([i, j, k]))
            # Query the KDTree for samples within the cell:
            indices = self.kdtree.query_ball_point(centroid, r=np.sqrt(3)/2*self.cell_size[0])
            # Filter out samples that have already been assigned to cells
            indices = [index for index in indices if index not in assigned_samples]
            samples = self.samples_df.iloc[indices]
            self.cells[i, j, k] = GridCell(centroid, samples)
            # Add the indices of the assigned samples to the set
            assigned_samples.update(indices)
        print(f"Grid initialization complete. {len(assigned_samples)} samples assigned")

    def initialize_irregular_grid(self, cells_df):
        assigned_samples = set()
        for index, row in tqdm(cells_df.iterrows(), total=cells_df.shape[0], desc=f'Processing cells'):
            centroid = (row['x'], row['y'], row['z'])
            try:
                domain = row['Domain']
                self.domained = True
            except KeyError:
                pass
            # Calculate the cell indices for the given position
            i, j, k = tuple(min(int((pos - self.lower_bounds[d]) / self.cell_size[d]), self.grid_dimension[d] - 1) for d, pos in enumerate(centroid))
            # Query the KDTree for samples within the cell
            indices = self.kdtree.query_ball_point(centroid, r=np.sqrt(3)/2*self.cell_size[0])
            # Filter out samples that have already been assigned to cells
            indices = [index for index in indices if index not in assigned_samples]
            samples = self.samples_df.iloc[indices]
            #samples = self.samples_df[samples_df['grid_indices'] == (i, j, k)] #Alternative to KDTree
            self.cells[i, j, k] = GridCell(centroid, samples)
            self.cells[i, j, k].domain = domain
            assigned_samples.update(indices)
        print(f"Grid initialization complete. {len(assigned_samples)} samples assigned")

    def get_cell(self, position):
        #Return a cell for a given position
        indices = self.get_indices(position)

        # Return the cell at the calculated indices
        return self.cells[indices]

    def get_indices(self,position):
        # Return the cell indices for a given position
        return tuple(min(int((pos - self.lower_bounds[d]) / self.cell_size[d]), self.grid_dimension[d] - 1) for d, pos in enumerate(position))

class GridCell:
    def __init__(self, position, samples,ant_capacity=100):
        self.position = position
        #self.samples = samples
        self.best_prediction = None #lesser difference between prediction and cell value
        self.best_coefficients = None
        self.best_direction=None
        self.best_xp=None
        # Initialize the value to the average of the samples if there are any
        self.value = samples['Value'].mean() if not samples.empty else None
        try:
            self.domain = samples['Domain'].mode()[0] if not samples.empty else None
        except KeyError:
            # If there is no 'Domain' column in the samples, set the domain to None
            self.domain = None
        self.has_samples = not samples.empty
        self.num_ants=0
        self.ant_capacity=ant_capacity

        self.tracked_value=None #value of the sample being tracked with pheromone
        self.pheromone=0

    def update_best_prediction(self, prediction, ant):

        coefficients = ant.coefficients

        # Calculate the absolute difference between the prediction and the actual value
        difference = abs(prediction - self.value)

        # If the prediction is closer to the actual value than the best prediction so far, update the best prediction and coefficients
        if self.best_prediction is None or difference < self.best_prediction:
            self.best_prediction = difference
            self.best_coefficients = coefficients
            self.best_direction = ant.direction
            ant.xp+=1
            self.best_xp=ant.xp
        else:
            ant.xp= max(ant.xp-1,0) #ant looses experience

class Ant:
    def __init__(self, position, coefficients, tolerance, cell_size, lower_bounds, upper_bounds, domain=None):
        self.position = position
        self.last_position = position
        self.coefficients = coefficients
        self.domain = domain
        self.tolerance = tolerance
        self.cell_size = cell_size
        self.lower_bounds = lower_bounds
        self.upper_bounds = upper_bounds
        self.direction = (0,0,0) #relative direction the ant is moving
        self.xp=0 #experience points
        self.last_sample_value=None
        self.max_pheromone=max([int((upper_bounds[i]-lower_bounds[i])/cell_size[i]) for i in range(3)])

    def move(self, grid):
        # Create a list to store the positions of empty cells and cells that are out of the ant's prediction tolerance
        second_choice_cells = set()
        out_of_grid_cells = set()
        out_of_domain_cells = set()
        occupied_cells = set()

        current_cell = grid.get_cell(self.position)

        follow_leader=True
        if follow_leader:
            #the ant changes direction towards more experienced ant
            if current_cell.best_direction is not None and current_cell.best_xp > self.xp: #!=0:
                cLeader=current_cell.best_xp/(current_cell.best_xp+self.xp)
                cFollower = self.xp/(current_cell.best_xp+self.xp)
                #The most difference of experience between leader and follower, the closest the new direction to that of the leader
                new_direction = tuple(round_half_up(cFollower*self.direction[i]+cLeader*current_cell.best_direction[i]) for i in range(3))
                neighbor_position = tuple(self.position[i] + new_direction[i]*self.cell_size[i] for i in range(3))
                neighbor_cell = grid.get_cell(neighbor_position)
                if neighbor_cell is None:
                    grid.out_of_grid_cells.add(tuple(neighbor_position))
                    out_of_grid_cells.add(tuple(neighbor_position))
                elif grid.domained and neighbor_cell.domain != self.domain:
                    out_of_domain_cells.add(tuple(neighbor_position))
                elif neighbor_cell.num_ants >= neighbor_cell.ant_capacity:
                    occupied_cells.add(tuple(neighbor_position))
                else: #neighbor_cell is not None:
                    self.last_position = self.position
                    self.position = neighbor_position
                    self.direction = new_direction
                    current_cell.num_ants-=1
                    neighbor_cell.num_ants+=1
                    return

        while True:
            # Choose a random offset
            offset = random.choice(OFFSETS)

            # Compute the position of the neighboring cell
            neighbor_position = tuple(self.position[i] + offset[i]*self.cell_size[i] for i in range(3))
            neighbor_cell = grid.get_cell(neighbor_position) # Get the cell at the new position

            # Check if the new position is a known out of grid cell:
            if (neighbor_position in grid.out_of_grid_cells):
                out_of_grid_cells.add(tuple(neighbor_position))

            # Check if the new position is a None cell:
            elif neighbor_cell is None:
                grid.out_of_grid_cells.add(tuple(neighbor_position))
                out_of_grid_cells.add(tuple(neighbor_position))

            # Check if the new position is inside the grid and domain
            elif all(self.lower_bounds[i] <= neighbor_position[i] < self.upper_bounds[i] for i in range(3)):

                # If the neighbor cell's domain doesn't match the ant domain
                if(grid.domained and (neighbor_cell.domain != self.domain)):
                    out_of_domain_cells.add(tuple(neighbor_position))

                # if the neighbor cell is full:
                elif neighbor_cell.num_ants >= neighbor_cell.ant_capacity:
                        occupied_cells.add(tuple(neighbor_position))

                # If the neighbor cell's value is None or its value is close enough to the prediction
                elif neighbor_cell.value is None or abs(self.predict(grid,neighbor_cell) - neighbor_cell.value) < self.tolerance:
                    self.last_position = self.position
                    self.position = neighbor_position
                    self.direction = offset
                    current_cell.num_ants-=1
                    neighbor_cell.num_ants+=1
                    return

                # If the neighbor cell's value is close enough to current cell's value
                elif abs(current_cell.value - neighbor_cell.value) < self.tolerance:
                    self.last_position = self.position
                    self.position = neighbor_position
                    self.direction = offset
                    current_cell.num_ants-=1
                    neighbor_cell.num_ants+=1
                    return

                # If the neighbor cell's value is not close to the ant's prediction or the current cell, add it to the list of second choice cells
                else:
                    second_choice_cells.add(tuple(neighbor_position))
            else:
                grid.out_of_grid_cells.add(tuple(neighbor_position))
                out_of_grid_cells.add(tuple(neighbor_position))

            # If all neighboring cells have been checked, move to a random available cell
            if len(second_choice_cells) + len(out_of_grid_cells) + len(out_of_domain_cells) +len(occupied_cells) == len(OFFSETS):
                if second_choice_cells:  # Check if the list is not empty
                    neighbor_position = random.choice(list(second_choice_cells))
                    neighbor_cell = grid.get_cell(neighbor_position)
                    self.last_position = self.position
                    self.position = neighbor_position
                    self.direction = tuple(self.position[i] - current_cell.position[i] for i in range(3))
                    current_cell.num_ants-=1
                    neighbor_cell.num_ants+=1
                return

    def predict(self, grid, cell):
        # Get the indices of the current cell
        i, j, k = tuple(int((pos - grid.lower_bounds[d]) / grid.cell_size[d]) for d, pos in enumerate(self.position))

        #Initialize the prediction to the current cell's value times the last coefficient
        #Including the current cell in the prediction of its own value makes the interpolator converge to 0 too quickly away from samples
        #Not including the current cell in the prediction makes the interpolator to return values too high away from samples
        #self_inclusion parameter aims to calibrate this two behaviors into something in the middle.
        self_inclusion=0.1 #probability of a cell being included in the prediction of it's own value
        if random.random() > self_inclusion:
            prediction = 0 #If current position value is not considered
            valid_coefficients = [] #If current position value is not considered
        else:
            try:
                prediction = self.coefficients[-1] * cell.value if cell.value is not None else 0
                pass
            except AttributeError:
                print(f"\nPrediction Error")
                print(f"Current cell indices: {(i,j,k)}. Current cell position: {cell if(cell is None) else cell.position}")
                print(f"Expected ant cell indices: {grid.get_indices(self.position)}. Current ant position: {self.position}")
                exit()

            # Initialize a list to store the coefficients of the cells that have a value
            valid_coefficients = [self.coefficients[-1]] #if cell.value is not None else []

        # Loop over the offsets to get the neighboring cells
        for idx, offset in enumerate(OFFSETS):
            # Compute the indices of the neighboring cell
            neighbor_i = i + offset[0]
            neighbor_j = j + offset[1]
            neighbor_k = k + offset[2]

            # Check if the indices are within the grid
            if (0 <= neighbor_i < grid.grid_dimension[0] and
                0 <= neighbor_j < grid.grid_dimension[1] and
                0 <= neighbor_k < grid.grid_dimension[2]):

                # Get the neighboring cell
                neighbor_cell = grid.cells[neighbor_i, neighbor_j, neighbor_k]

                if neighbor_cell is not None:

                    # Check is the cell is within the domain
                    if (not grid.domained) or(self.domain == neighbor_cell.domain):

                        # If the cell has a value, add the weighted pheromone level of the cell to the prediction and add the coefficient to the list of valid coefficients
                        if neighbor_cell.value is not None:
                            #if neighbor_cell.value == 0: print("Zero value cell")
                            prediction += self.coefficients[idx] * neighbor_cell.value
                            valid_coefficients.append(self.coefficients[idx])
            else:
                out_position = tuple(grid.lower_bounds[d] + (idx + 0.5) * grid.cell_size[d] for d, idx in enumerate([neighbor_i, neighbor_j,  neighbor_k]))
                grid.out_of_grid_cells.add(out_position)

        # Normalize the prediction by the sum of the valid coefficients
        if valid_coefficients:
            prediction /= sum(valid_coefficients)
        # Return the prediction
        return prediction

    def update_coefficients(self, best_coefficients):
        # If the cell has no best coefficients yet, do nothing
        if best_coefficients is None:
            return

        # Calculate the new coefficients as the average of the ant's current coefficients and the best coefficients of the cell
        self.coefficients = [(self.coefficients[i] + best_coefficients[i]) / 2 for i in range(27)]

    def update_trail(self,grid):
        #Update pheromone trail, in order to generate a pheromone trail between samples with simmilar values

        last_cell = grid.get_cell(self.last_position)
        current_cell = grid.get_cell(self.position)
        tolerance = self.tolerance

        if current_cell.has_samples:
            self.last_sample_value = current_cell.value
            current_cell.tracked_value = current_cell.value
            current_cell.pheromone=self.max_pheromone
            #return

        if self.last_sample_value is None:
            print(f"Error: ant has no last sample value")

        if current_cell.position != last_cell.position:

            dcurrent=abs(current_cell.value - self.last_sample_value) #if current cell has samples, dcurrent is 0
            dlast=abs(last_cell.value - self.last_sample_value)

            #This is a patch, should be better than this
            if dlast <= tolerance and last_cell.tracked_value is None:
                last_cell.tracked_value = self.last_sample_value
                last_cell.pheromone = self.max_pheromone//2

            if max(dlast,dcurrent) <= tolerance:

                if last_cell.pheromone==0:
                    print(f"Warning: last visited cell in track has no pheromone assigned. Value: {last_cell.value}. Tracked value: {last_cell.tracked_value}.")

                current_cell.tracked_value = self.last_sample_value
                if dcurrent<dlast: #current cell is closer to last sample value
                    current_cell.pheromone=min(last_cell.pheromone + 1,self.max_pheromone)
                else:
                    current_cell.pheromone=max(last_cell.pheromone - 1,0)

class Interpolator:
    def __init__(self, ants, grid, background=0, fade = 0.1):
        self.ants = ants
        self.grid = grid
        self.background=background

    def run(self, num_iterations):
        for _ in tqdm(range(num_iterations),desc='Interpolating'):

            # Shuffle the ants
            random.shuffle(self.ants)

            for ant in self.ants:

                ant.update_trail(self.grid)

                #Ant move to a new cell
                ant.move(self.grid)
                antcell=self.grid.get_cell(ant.position)
                if antcell is None:
                    print("\nError: Ant moved to None cell")
                    exit()

                #Ant predicts the value of its current cell
                cell = self.get_cell(ant.position)
                prediction = ant.predict(self.grid, cell)  # Pass the Grid object here

                #And updates the cell based on it's prediction
                if cell.has_samples:  # Check if the cell has samples
                    cell.update_best_prediction(prediction, ant)
                    ant.update_coefficients(cell.best_coefficients)
                else:
                    cell.value = max(prediction,self.background)  # Set the cell's value to the prediction for cells without samples
                    if (cell.best_xp is None or (ant.xp > cell.best_xp)):
                        cell.best_xp = ant.xp
                        cell.best_direction = ant.direction

            #self.evaporate_pheromone()

    def get_cell(self, position):
      # Calculate the cell indices for the given position
      indices = tuple(min(int((pos - lower_bounds[d]) / cell_size[d]), self.grid.grid_dimension[d] - 1) for d, pos in enumerate(position))

      # Return the cell at the calculated indices
      return self.grid.cells[indices]

    def evaporate_pheromone(self):
        # Implement the logic for evaporating the pheromone here
        for plane in self.grid.cells:
            for row in plane:
                for cell in row:
                    if cell is not None and cell.value is not None and not cell.has_samples:
                        cell.value = max(cell.value - 0.1, self.background)

#Input parameters
SamplesFile="/content/Samples_CNN 03.csv"
CellsFile = None
base_name, extension = os.path.splitext(SamplesFile)
if CellsFile is not None: base_name=base_name + "_Domained"

#Grid parameters
cell_size = (10,10,10)
grid_rebuild=True

# Interpolation parameters
nAnts =2000  # The desired number of ants
tolerance =  5
iterations = 4000
#background = samples_df['Value'].mean()
background=0 #or any constant value you want
fillbackground=False

#Output parameters
plotscatter=False
download = True

#Load and validate samples
print("Loading samples from file " + SamplesFile + "...")
samples_df = pd.read_csv(SamplesFile,delimiter=",")
print(f'Samples loaded: {len(samples_df)}')
samples_df['Value'] = pd.to_numeric(samples_df['Value'], errors='coerce') #Converts column Value to numeric
samples_df = samples_df.dropna() #remove rows with NaN values
print(f'Valid (numeric) samples: {len(samples_df)}')
samples_df.to_csv("SamplesCheck.csv", index=False)

# Calculate the grid bounds
inputcells_df=None
if(CellsFile is None):

    lower_bounds = samples_df[['x', 'y', 'z']].min().values - 0.5 * np.array(cell_size)
    upper_bounds = samples_df[['x', 'y', 'z']].max().values + 0.5 * np.array(cell_size)

else:
    print("Loading cells definition from file " + CellsFile + "...")
    inputcells_df = pd.read_csv(CellsFile,delimiter=";")
    print(f"Cells loaded: {len(inputcells_df)}")
    # Convert non-numeric values in the 'x', 'y', and 'z' columns to NaN
    inputcells_df['x'] = pd.to_numeric(inputcells_df['x'], errors='coerce')
    inputcells_df['y'] = pd.to_numeric(inputcells_df['y'], errors='coerce')
    inputcells_df['z'] = pd.to_numeric(inputcells_df['z'], errors='coerce')

    # Remove rows with NaN values in the 'x', 'y', or 'z' columns
    inputcells_df = inputcells_df.dropna(subset=['x', 'y', 'z'])
    print(f"Cells with valid (numeric) coordinates: {len(inputcells_df)}")

    subset_df = inputcells_df.head(100)
    subset_df = subset_df.sort_values(['x', 'y', 'z'])
    differences = subset_df[['x', 'y', 'z']].diff().abs() # Calculate the difference between consecutive rows
    cell_size_series = differences[differences > 0].min() # The cell size in each dimension is the minimum non-zero difference
    cell_size = tuple(cell_size_series) # Convert the Series to a tuple
    lower_bounds = inputcells_df[['x', 'y', 'z']].min().values - 0.5 * np.array(cell_size)
    upper_bounds = inputcells_df[['x', 'y', 'z']].max().values + 0.5 * np.array(cell_size)

print("Lower bounds: " + str(lower_bounds))
print("Upper bounds: " + str(upper_bounds))

# Calculate the grid size
grid_size = upper_bounds - lower_bounds

# Calculate the grid dimension
grid_dimension = [int(grid_size[i] / cell_size[i]) for i in range(3)]
total_cells = grid_dimension[0]*grid_dimension[1]*grid_dimension[2]

#Initialize the grid
print("Grid size: " + str(grid_size))
print("Grid dimension: " + str(grid_dimension))
print("Cells size: " + str(cell_size))
print(f"Total cells: {total_cells}")

print("Assigning cell indices to samples...")
samples_df['grid_indices'] = list(zip(
    (np.minimum((samples_df['x'] - lower_bounds[0]) / cell_size[0], grid_dimension[0] - 1)).astype(int),
    (np.minimum((samples_df['y'] - lower_bounds[1]) / cell_size[1], grid_dimension[1] - 1)).astype(int),
    (np.minimum((samples_df['z'] - lower_bounds[2]) / cell_size[2], grid_dimension[2] - 1)).astype(int)
))
print("Indices assigned")

#Grid initialization
if (grid_rebuild):
    print("Initializing grid...")
    grid = Grid(grid_dimension, lower_bounds, cell_size, samples_df)
    if(CellsFile is None):
        grid.initialize_regular_grid()
    else:
        grid.initialize_irregular_grid(inputcells_df)

# Create a DataFrame for the grid cells
InitGrid_df = pd.DataFrame([(cell.position[0], cell.position[1], cell.position[2], cell.value, cell.domain, cell.num_ants) for cell in grid.cells.flatten() if cell is not None and cell.value is not None], columns=['x', 'y', 'z', 'Value','Domain', 'Num_Ants'])
InitGrid_df['size'] = InitGrid_df.apply(lambda row: 20 if grid.cells[int((row['x'] - lower_bounds[0]) / cell_size[0]),
                                                              int((row['y'] - lower_bounds[1]) / cell_size[1]),
                                                              int((row['z'] - lower_bounds[2]) / cell_size[2])].has_samples else 8, axis=1)
InitGrid_df.to_csv('GridBeforeInterpolation.csv', index=False)

del inputcells_df
gc.collect()

# Get the list of cells that contain samples
sample_cells = [cell for cell in grid.cells.flatten() if cell is not None and cell.value is not None]
print(f'Cells with samples: {len(sample_cells)}')

# Choose random cells for the ants
ant_cells = np.random.choice(sample_cells, size=nAnts, replace=True)

# Initialize the ants at the centroids of the chosen cells
ants = [Ant(cell.position, np.random.dirichlet(np.ones(27),size=1)[0], tolerance, cell_size, lower_bounds, upper_bounds, cell.domain) for cell in ant_cells]
print(f'Ants: {len(ants)}')
for ant in ants:
    grid.get_cell(ant.position).num_ants+=1

# Create a DataFrame from the list of ant coordinates
ants_df = pd.DataFrame([ant.position for ant in ants], columns=['x', 'y', 'z'])

# Save the DataFrame as a CSV file
ants_df.to_csv('ants_Initial_coordinates.csv', index=False)

# Check if there are enough sample cells for the number of ants
if len(sample_cells) < nAnts:
    print(f"Warning: There are only {len(sample_cells)} cells with samples, but {nAnts} ants are requested. Some ants will be placed in the same cell.")

print(f"Tolerance: {tolerance}")
print(f"Background value: {background}")

# Create the interpolator and run it
print("Starting interpolation...")
interpolator = Interpolator(ants, grid,background) # Interpolator(ants, grid) uses a background value of 0 as default
interpolator.run(num_iterations=iterations)
print("Interpolation complete.")

# Create a DataFrame for the grid cells
cells_df = pd.DataFrame([(cell.position[0], cell.position[1], cell.position[2], cell.value, cell.domain, cell.num_ants, cell.best_xp, cell.tracked_value, cell.pheromone) for cell in grid.cells.flatten() if cell is not None and cell.value is not None], columns=['x', 'y', 'z', 'Value','Domain', 'Num_Ants', 'Best_XP', 'Tracked_Value', 'Pheromone'])
cells_df['size'] = cells_df.apply(lambda row: 20 if grid.cells[int((row['x'] - lower_bounds[0]) / cell_size[0]),
                                                              int((row['y'] - lower_bounds[1]) / cell_size[1]),
                                                              int((row['z'] - lower_bounds[2]) / cell_size[2])].has_samples else 8, axis=1)
cells_visited = len(cells_df)
print(f'Cells visited: {cells_visited} ({round(cells_visited/total_cells,4)*100}%)')

#Assign background value to unvisited cells
if fillbackground:
    print("Assigning background value to unvisited cells...")
    cells_df['Value'] = cells_df['Value'].fillna(background)

# Save the DataFrame to a CSV file
cells_file = f"{base_name}_{cell_size[0]}x{cell_size[1]}x{cell_size[2]}_a{nAnts}_t{tolerance}_b{background}_i{iterations}.csv"
cells_df.to_csv(cells_file, index=False)
print("Cells saved in " + cells_file)

if plotscatter:

    # Create a 3D scatter plot of the interpolated values
    fig1 = px.scatter_3d(cells_df, x='x', y='y', z='z', color='Value', size='size',color_continuous_scale=['green','yellow','orange','red'])
    fig1.update_traces(marker=dict(line=dict(width=2, color='Black')))
    fig1.update_layout(scene=dict(
        xaxis=dict(range=[lower_bounds[0], upper_bounds[0]], tickmode='linear', tick0=lower_bounds[0], dtick=cell_size[0],backgroundcolor="black"),
        yaxis=dict(range=[lower_bounds[1], upper_bounds[1]], tickmode='linear', tick0=lower_bounds[1], dtick=cell_size[1],backgroundcolor="black"),
        zaxis=dict(range=[lower_bounds[2], upper_bounds[2]], tickmode='linear', tick0=lower_bounds[2], dtick=cell_size[2],backgroundcolor="black"),
        camera=dict(projection=dict(type='orthographic'))
    ))
    fig1.show()

if download: files.download(cells_file)

Loading samples from file /content/Samples_CNN 03.csv...
Samples loaded: 47306
Valid (numeric) samples: 47306
Lower bounds: [3.66426255e+05 6.99858503e+06 6.82176549e+02]
Upper bounds: [3.67541590e+05 7.00301491e+06 1.50164926e+03]
Grid size: [1115.3352    4429.879      819.4727093]
Grid dimension: [111, 442, 81]
Cells size: (10, 10, 10)
Total cells: 3974022
Assigning cell indices to samples...
Indices assigned
Initializing grid...


Initializing empty grid...:   0%|          | 0/111 [00:00<?, ?it/s]

Assigning samples to cells...: 0it [00:00, ?it/s]

Grid initialization complete. 47290 samples assigned
Cells with samples: 9940
Ants: 2000
Tolerance: 5
Background value: 0
Starting interpolation...


Interpolating:   0%|          | 0/4000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
cells_file = f"{base_name}_{cell_size[0]}x{cell_size[1]}x{cell_size[2]}_a{nAnts}_t{tolerance}_b{background}_i{iterations}.csv"
cells_df.to_csv(cells_file, index=False)

In [ ]:
import pandas as pd
import plotly.express as px

# Read the CSV data
SamplesFile="/content/Samples.csv"
df = pd.read_csv(SamplesFile,delimiter=";")

# Create a 3D scatter plot
fig = px.scatter_3d(df, x='x', y='y', z='z', color='Value')

# Show the plot
fig.show()